Import credentials

In [41]:
import os
import json

path_private_key = '/content/drive/MyDrive/Intro_to_big_data/zippy-athlete-416011-b9662b3c6922.json'

with open(path_private_key, 'r') as json_file:
    data = json.load(json_file)

for key, value in data.items():
    os.environ[key] = str(value)

In [43]:
from google.oauth2 import service_account

SERVICE_ACCOUNT_KEY_FILE = path_private_key

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_KEY_FILE)

Empty table

In [44]:
from google.cloud import bigquery

project_name = os.environ['project_id']
client = bigquery.Client(credentials=credentials, project=project_name)
dataset = client.dataset("lab_3")
table_ref = dataset.table("task_1_empty_table")

schema = [
    bigquery.SchemaField("product_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("price", "FLOAT", mode="NULLABLE"),
]

table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table zippy-athlete-416011.lab_3.task_1_empty_table


CSV to BigQuery table

In [45]:
from google.cloud import bigquery
import pandas as pd
import numpy as np


project_name = os.environ['project_id']
datase_name = "lab_3"
table_name = "task_1_from_pandas"

client = bigquery.Client(credentials=credentials, project=project_name)

table_id = f"{project_name}.{datase_name}.{table_name}"

df = pd.read_csv(r"/content/drive/MyDrive/Intro_to_big_data/apple_quality.csv")

pandas_to_bigquery_type_mapping = {
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'object': 'STRING',
}

schema = [
    bigquery.SchemaField(column_name, pandas_to_bigquery_type_mapping[str(df.dtypes[column_name])], mode="NULLABLE")
    for column_name in df.columns
]

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE"
)


job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)
job.result()

table = client.get_table(table_id)
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 4001 rows and 9 columns to zippy-athlete-416011.lab_3.task_1_from_pandas


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
